In [1]:
%matplotlib inline
import numpy as np
import matplotlib
import pandas as pd

# SVM支持向量机
***优点:*** 泛化错误率低，计算开销不大，结果易于理解  
***缺点:*** 对于参数调节和核函数的选择敏感，原始分类器不加修改仅适用于处理二分类问题  
***适用数据类型:*** 数值型和标称型数据

***支持向量(support vector)***就是离分隔超平面(separating hyperplane)最近的哪些点。模型在最大化支持向量到分隔面的距离。
### 寻找最大间隔
分隔超平面: $\mathbf{w^Tx} + b$  
点$A$到分隔超平面的距离$|\mathbf{w^TA} + b|/||\mathbf{w}||$
###### 分类器求解的最优化
海维赛德阶跃函数（单位阶跃函数）: $sign(u) = \left\{\begin{matrix}1 & u\geq 0 \\ -1 & u< 0 \end{matrix}\right.$  
分类器输出: $sign(\mathbf{w^Tx} + b)$  
间隔(函数间隔): $label\cdot (\mathbf{w^Tx} + b)$  
原始问题：
* 优化问题: $arg\; \underset{w,b}{\max}\left\{\underset{n}{min}(label\cdot (\mathbf{w^Tx}+b))\right\}$
* 约束条件: $label\cdot (\mathbf{w^Tx}+b)\geq1.0$

拉格朗日对偶问题:
* 优化问题: $\underset{\alpha}{\max}\left [ \sum_{i=1}^{m}\alpha - \frac{1}{2}\sum_{i,j=1}^{m}label^{(i)}\cdot label^{(j)}\cdot \left \langle x^{(i)},x^{(j)} \right \rangle \right ]$ _尖括号表示内积_
* 约束条件: $\left\{\begin{matrix}\alpha\geq0\\ \sum_{i=1}^{m}\alpha_i\cdot label^{(i)} = 0\end{matrix}\right.$

松弛变量(slack variable) 允许数据点处于错误的一侧
* 约束条件(对偶形式): $\left\{\begin{matrix}C\geq\alpha\geq0\\ \sum_{i=1}^{m}\alpha_i\cdot label^{(i)} = 0\end{matrix}\right.$

### SMO 算法
每次循环选择两个alpha进行优化处理，一旦找到一对合适的alpha，那么增大其中的一个同时减小另外一个（为保证$ \sum\alpha_i\cdot label^{(i)} = 0$）。 所谓合适:
* 两个alpha必须要在间隔边界之外
* 这两个alpha还没有进行过区间处理或者不在边界上

测试数据

In [2]:
def load_data_set(file_name):
    data_set = pd.read_csv(file_name, header=None, names=['x1','x2','label'], sep='\t', engine='python')
    return data_set[['x1','x2']].values, data_set['label'].values
data_arr, label_arr = load_data_set('data/testSet_SVM.txt')

###### 简化SMO
```
创建一个alpha向量并将其初始化为0
当迭代次数小于最大迭代次数时(外循环):
  对数据集中的每个数据向量(内循环):
    如果数据向量可以被优化:
      随机选择另外一个数据向量
      同时优化这两个向量
      如果两个向量都不能被优化，退出内循环
  如果所有向量都没被优化，增加迭代数目，继续下一次循环
```

辅助函数

In [3]:
# 随机选择
def select_jrand(i, m):
    j = i
    while j==i:
        j = int(np.random.uniform(0,m))
    return j
# 调整大于h或小于l的alpha值
def clip_alpha(aj, H, L):
    if aj > H:
        aj = H
    if L > aj:
        aj = L
    return aj

简化版SMO算法

In [4]:
def smo_simple(data_mat_in, class_labels, C, toler, max_iter):
    '''
    :param toter 容错率
    '''
    data_matrix, label_mat = np.mat(data_mat_in), np.mat(class_labels).transpose()
    b = 0
    m,n = np.shape(data_matrix)
    alphas = np.mat(np.zeros((m, 1)))
    _iter = 0
    while _iter < max_iter:
        alpha_pairs_changed = 0
        for i in range(m):
            f_x_i = np.float(np.multiply(alphas, label_mat).T * (data_matrix*data_matrix[i,:].T)) + b
            e_i = f_x_i - np.float(label_mat[i])
            if (label_mat[i] * e_i < -toler and alphas[i] < C) or \
               (label_mat[i] * e_i > toler and alphas[i] > 0):
                # 如果alpha可以更改，进入优化过程
                j = select_jrand(i, m) #随机选择第二个alpha
                f_x_j = np.float(np.multiply(alphas, label_mat).T * (data_matrix*data_matrix[j,:].T)) + b
                e_j = f_x_j - np.float(label_mat[j])
                alpha_i_old = alphas[i].copy()
                alpha_j_old = alphas[j].copy()
                #保证alpha在C与0之间
                if label_mat[i] != label_mat[j]:
                    L = max(0, alphas[j] - alphas[i])
                    H = min(C, C + alphas[j] - alphas[i])
                else:
                    L = max(0, alphas[j] + alphas[i] - C)
                    H = min(C, alphas[j] + alphas[i])
                if L == H:
                    print("L==H")
                    continue
                eta = 2.0 * data_matrix[i, :] * data_matrix[j,:].T - data_matrix[i,:]*data_matrix[i,:].T \
                     - data_matrix[j,:]*data_matrix[j,:].T
                if eta >= 0:
                    print("eta >= 0")
                    continue
                
                alphas[j] -= label_mat[j] * (e_i - e_j) / eta
                alphas[j] = clip_alpha(alphas[j], H, L)
                if np.abs(alphas[j] - alpha_j_old) < 0.00001:
                    print("j not moving enough")
                    continue
                # 同时修改i, j但是方向相反
                alphas[i] += label_mat[j] * label_mat[i] * (alpha_j_old - alphas[j])
                # 设置常数项
                b1 = b - e_i - label_mat[i]*(alphas[i] - alpha_i_old) * data_matrix[i,:]*data_matrix[i,:].T \
                    - label_mat[j]*(alphas[j] - alpha_j_old) * data_matrix[i,:]*data_matrix[j,:].T
                b2 = b - e_j - label_mat[i]*(alphas[i] - alpha_i_old) * data_matrix[i,:]*data_matrix[j,:].T \
                    - label_mat[j]*(alphas[j] - alpha_j_old) * data_matrix[j,:]*data_matrix[j,:].T
                if 0 < alphas[i] and C > alphas[i]:
                    b = b1
                elif 0 < alphas[j] and C > alphas[j]:
                    b = b2
                else:
                    b = (b1 + b2) / 2.0
                alpha_pairs_changed += 1
                print("iter: %d i:%d, pair changed %d"%(_iter, i, alpha_pairs_changed))
        if alpha_pairs_changed == 0:
            _iter += 1
        else:
            _iter = 0
        print("iteration number: %d"%_iter)
    return b, alphas

测试

In [5]:
b, alphas = smo_simple(data_arr, label_arr, 0.6, 0.001, 40)

L==H
iter: 0 i:1, pair changed 1
iter: 0 i:2, pair changed 2
iter: 0 i:4, pair changed 3
L==H
L==H
j not moving enough
iter: 0 i:10, pair changed 4
iter: 0 i:17, pair changed 5
L==H
iter: 0 i:27, pair changed 6
L==H
L==H
j not moving enough
iter: 0 i:43, pair changed 7
L==H
L==H
L==H
j not moving enough
L==H
j not moving enough
j not moving enough
L==H
L==H
j not moving enough
j not moving enough
L==H
L==H
j not moving enough
L==H
L==H
L==H
L==H
j not moving enough
L==H
iteration number: 0
L==H
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
L==H
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
L==H
j not moving enough
j not moving enough
L==H
L==H
iter: 0 i:32, pair changed 1
j not moving enough
j not moving enough
L==H
iter: 0 i:54, pair changed 2
L==H
j not moving enough
j not moving enough
iter: 0 i:69, pair changed 3
L==

iteration number: 0
j not moving enough
j not moving enough
j not moving enough
j not moving enough
iteration number: 1
j not moving enough
j not moving enough
j not moving enough
j not moving enough
iteration number: 2
j not moving enough
j not moving enough
j not moving enough
j not moving enough
iteration number: 3
j not moving enough
j not moving enough
j not moving enough
j not moving enough
iteration number: 4
j not moving enough
j not moving enough
iter: 4 i:52, pair changed 1
j not moving enough
iteration number: 0
j not moving enough
j not moving enough
j not moving enough
j not moving enough
iteration number: 1
j not moving enough
j not moving enough
j not moving enough
j not moving enough
iteration number: 2
iter: 2 i:17, pair changed 1
j not moving enough
j not moving enough
j not moving enough
j not moving enough
iteration number: 0
j not moving enough
iter: 0 i:52, pair changed 1
j not moving enough
iteration number: 0
j not moving enough
j not moving enough
j not moving 

j not moving enough
j not moving enough
j not moving enough
iteration number: 17
j not moving enough
j not moving enough
j not moving enough
iteration number: 18
j not moving enough
j not moving enough
j not moving enough
iteration number: 19
j not moving enough
j not moving enough
j not moving enough
iteration number: 20
j not moving enough
j not moving enough
j not moving enough
iteration number: 21
j not moving enough
j not moving enough
j not moving enough
iteration number: 22
j not moving enough
j not moving enough
j not moving enough
iteration number: 23
j not moving enough
j not moving enough
j not moving enough
iteration number: 24
j not moving enough
j not moving enough
j not moving enough
iteration number: 25
j not moving enough
j not moving enough
j not moving enough
iteration number: 26
j not moving enough
j not moving enough
j not moving enough
iteration number: 27
j not moving enough
j not moving enough
j not moving enough
iteration number: 28
j not moving enough
j not mo

In [6]:
b

matrix([[-3.8219411]])

In [7]:
alphas[alphas>0]

matrix([[0.07812545, 0.12178204, 0.15397027, 0.13983638, 0.21404138]])

In [8]:
# 支持向量个数
np.shape(alphas[alphas>0])

(1, 5)

In [9]:
# 支持向量
for i in range(100):
    if alphas[i]>0.0:
        print(data_arr[i], label_arr[i])

[4.658191 3.507396] -1
[ 3.457096 -0.082216] -1
[ 2.893743 -1.643468] -1
[ 5.286862 -2.358286] 1
[6.080573 0.418886] 1


##### 完整Platt SMO算法
外循环选择第一个alpha: 两种模式交替
* 所有数据集上单遍遍历
* 在非边界alpha中实现单遍遍历（不等于边界0或C的alpha值）建立alpha值得列表， 跳过已知的不会改变的alpha

内循环选择第二个alpha值：最大化步长  建立缓存保存误差值，选择使步长($E_i - E_j$)最大的alpha值

缓存辅助函数

In [10]:
class OptStruct:
    def __init__(self, data_mat_in, class_labels, C, toler, *others):
        self.X = data_mat_in
        self.label_mat = class_labels
        self.C = C
        self.tol = toler
        self.m = np.shape(data_mat_in)[0]
        self.alphas = np.mat(np.zeros((self.m, 1)))
        self.b = 0
        self.e_cache = np.mat(np.zeros((self.m, 2))) #误差缓存, (是否有效，e_k)
        
def calc_e_k(o_s, k):
    f_x_k = float(np.multiply(o_s.alphas, o_s.label_mat).T * (o_s.X*o_s.X[k,:].T)) + o_s.b
    return f_x_k - float(o_s.label_mat[k])
    
def select_j(i, o_s, e_i):
    max_k, max_delta_e, e_j = -1, 0, 0
    o_s.e_cache[i] = [1,e_i]
    valid_e_cache_list = np.nonzero(o_s.e_cache[:,0].A)[0]
    if len(valid_e_cache_list) > 1:
        for k in valid_e_cache_list:
            if k==i:
                continue
            e_k = calc_e_k(o_s, k)
            delta_e = abs(e_i - e_k)
            if delta_e > max_delta_e:
                # 选择最大步长的j
                max_k, max_delta_e, e_j = k, delta_e, e_k
        return max_k, e_j
    else:
        j = select_jrand(i, o_s.m)
        e_j = calc_e_k(o_s, j)
        return j, e_j
        
def update_e_k(o_s, k):
    e_k = calc_e_k(o_s, k)
    o_s.e_cache[k] = [1, e_k]

内循环

In [11]:
def inner_l(i, o_s):
    e_i = calc_e_k(o_s, i)
    if (o_s.label_mat[i] * e_i < -o_s.tol and  o_s.alphas[i] < o_s.C) or \
       (o_s.label_mat[i] * e_i > o_s.tol and  o_s.alphas[i] > 0):
        j, e_j = select_j(i, o_s, e_i) #第二个alpha选择的启发式方法
        alpha_i_old, alpha_j_old = o_s.alphas[i].copy(), o_s.alphas[j].copy()
        if o_s.label_mat[i] != o_s.label_mat[j]:
            L = max(0, o_s.alphas[j] - o_s.alphas[i])
            H = min(o_s.C, o_s.C + o_s.alphas[j] - o_s.alphas[i])
        else:
            L = max(0, o_s.alphas[i] + o_s.alphas[j] - o_s.C)
            H = min(o_s.C, o_s.alphas[j] + o_s.alphas[i])
        if L==H:
            print("L==H")
            return 0
        eta = 2.0 * o_s.X[i, :] * o_s.X[j,:].T - o_s.X[i,:]*o_s.X[i,:].T -o_s.X[j,:]*o_s.X[j,:].T
        if eta >= 0:
            print("eta >= 0")
            return 0
        o_s.alphas[j] -= o_s.label_mat[j] * (e_i - e_j) / eta
        o_s.alphas[j] = clip_alpha(o_s.alphas[j], H, L)
        update_e_k(o_s, j) # 更新误差缓存
        if abs(o_s.alphas[j] - alpha_j_old) < 0.00001:
            print("j not moving enough")
            return 0
        # 同时修改i, j但是方向相反
        o_s.alphas[i] += o_s.label_mat[j] * o_s.label_mat[i] * (alpha_j_old - o_s.alphas[j])
        update_e_k(o_s, i) #更新误差缓存
        # 设置常数项
        b1 = o_s.b - e_i - o_s.label_mat[i]*(o_s.alphas[i] - alpha_i_old) * o_s.X[i,:]*o_s.X[i,:].T \
            - o_s.label_mat[j]*(o_s.alphas[j] - alpha_j_old) * o_s.X[i,:]*o_s.X[j,:].T
        b2 = o_s.b - e_j - o_s.label_mat[i]*(o_s.alphas[i] - alpha_i_old) * o_s.X[i,:]*o_s.X[j,:].T \
                - o_s.label_mat[j]*(o_s.alphas[j] - alpha_j_old) * o_s.X[j,:]*o_s.X[j,:].T
        if 0 < o_s.alphas[i] and o_s.C > o_s.alphas[i]:
            o_s.b = b1
        elif 0 < o_s.alphas[j] and o_s.C > o_s.alphas[j]:
            o_s.b = b2
        else:
            o_s.b = (b1 + b2) / 2.0
        return 1
    else:
        return 0

外循环

In [12]:
def smo_p(data_mat_in, class_labels, C, toler, max_inter, k_tup = ('lin', 0)):
    o_s = OptStruct(np.mat(data_mat_in), np.mat(class_labels).transpose(), C, toler, k_tup)
    _iter = 0
    entire_set, alpha_pairs_changed = True, 0
    while _iter < max_inter and (alpha_pairs_changed > 0 or entire_set):
        alpha_pairs_changed = 0
        if entire_set:
            # 遍历所有的值
            for i in range(o_s.m):
                alpha_pairs_changed += inner_l(i, o_s)
                print("fullSet, iter: %d i: %d, pair changed %d" % (_iter, i, alpha_pairs_changed))
            _iter += 1
        else:
            #遍历非边界值
            non_bound_i_s = np.nonzero((o_s.alphas.A > 0) * (o_s.alphas.A < C))[0]
            for i in non_bound_i_s:
                alpha_pairs_changed += inner_l(i, o_s)
                print("non-bound, iter: %d i: %d, pair changed %d" % (_iter, i, alpha_pairs_changed))
            _iter += 1
        if entire_set:
            entire_set = False
        elif alpha_pairs_changed == 0:
            entire_set = True
        print("iteration number: %d" % _iter)
    return o_s.b, o_s.alphas

测试

In [13]:
b, alphas = smo_p(data_arr, label_arr, 0.6, 0.001, 40)

L==H
fullSet, iter: 0 i: 0, pair changed 0
L==H
fullSet, iter: 0 i: 1, pair changed 0
fullSet, iter: 0 i: 2, pair changed 1
L==H
fullSet, iter: 0 i: 3, pair changed 1
fullSet, iter: 0 i: 4, pair changed 2
fullSet, iter: 0 i: 5, pair changed 2
fullSet, iter: 0 i: 6, pair changed 2
j not moving enough
fullSet, iter: 0 i: 7, pair changed 2
L==H
fullSet, iter: 0 i: 8, pair changed 2
fullSet, iter: 0 i: 9, pair changed 2
L==H
fullSet, iter: 0 i: 10, pair changed 2
L==H
fullSet, iter: 0 i: 11, pair changed 2
L==H
fullSet, iter: 0 i: 12, pair changed 2
fullSet, iter: 0 i: 13, pair changed 2
L==H
fullSet, iter: 0 i: 14, pair changed 2
fullSet, iter: 0 i: 15, pair changed 2
fullSet, iter: 0 i: 16, pair changed 2
L==H
fullSet, iter: 0 i: 17, pair changed 2
fullSet, iter: 0 i: 18, pair changed 3
fullSet, iter: 0 i: 19, pair changed 3
fullSet, iter: 0 i: 20, pair changed 3
fullSet, iter: 0 i: 21, pair changed 3
j not moving enough
fullSet, iter: 0 i: 22, pair changed 3
L==H
fullSet, iter: 0 i: 23,

###### 预测

从alpha计算w

In [14]:
def calc_ws(alphas, data_arr, class_labels):
    X, label_mat = np.mat(data_arr), np.mat(class_labels).transpose()
    m, n = np.shape(X)
    w = np.zeros((n,1))
    for i in range(m):
        w += np.multiply(alphas[i]*label_mat[i], X[i,:].T)
    return w

In [15]:
ws = calc_ws(alphas, data_arr, label_arr)
ws

array([[ 0.65307162],
       [-0.17196128]])

预测

In [16]:
data_mat = np.mat(data_arr)
data_mat[0]*np.mat(ws) + b

matrix([[-0.92555695]])

In [17]:
label_arr[0]

-1

### 核函数

核技巧(kernel trick,也叫核变电 kernel substation): 将内积替换成核函数以实现将数据映射到高维空间

###### 径向基核函数 RBF
高斯版本RBF: $k(x,y) = exp\left (\frac{-\left \| x-y \right \|^2}{2\sigma ^2}\right )$ $\sigma$用于确定到达率(reach)或者说函数跌落到0到速度

核转换函数

In [18]:
def kernel_trans(X, A, k_tup):
    m, n = np.shape(X)
    K = np.mat(np.zeros((m,1)))
    if k_tup[0] == 'line':
        K = X * A.T #线性核函数
    elif k_tup[0] == 'rbf':
        for j in range(m):
            delta_row = X[j,:] - A
            K[j] = delta_row*delta_row.T
        K = np.exp(K / (-1*k_tup[1]**2)) #元素间除法
    else:
        raise NameError("Houston We Have a problem -- That Kernel is not recognized")
    return K

改造辅助函数

In [19]:
class OptStruct:
    def __init__(self, data_mat_in, class_labels, C, toler, k_tup):
        self.X = data_mat_in
        self.label_mat = class_labels
        self.C = C
        self.tol = toler
        self.m = np.shape(data_mat_in)[0]
        self.alphas = np.mat(np.zeros((self.m, 1)))
        self.b = 0
        self.e_cache = np.mat(np.zeros((self.m, 2))) #误差缓存, (是否有效，e_k)
        # 增加kernal变换
        self.K = np.mat(np.zeros((self.m, self.m)))
        for i in range(self.m):
            self.K[:, i] = kernel_trans(self.X, self.X[i,:], k_tup)
            
def calc_e_k(o_s, k):
    # 核函数替代内积
    f_x_k = float(np.multiply(o_s.alphas, o_s.label_mat).T * o_s.K[:, k]) + o_s.b
    return f_x_k - float(o_s.label_mat[k])

改造内循环

In [20]:
def inner_l(i, o_s):
    e_i = calc_e_k(o_s, i)
    if (o_s.label_mat[i] * e_i < -o_s.tol and  o_s.alphas[i] < o_s.C) or \
       (o_s.label_mat[i] * e_i > o_s.tol and  o_s.alphas[i] > 0):
        j, e_j = select_j(i, o_s, e_i) #第二个alpha选择的启发式方法
        alpha_i_old, alpha_j_old = o_s.alphas[i].copy(), o_s.alphas[j].copy()
        if o_s.label_mat[i] != o_s.label_mat[j]:
            L = max(0, o_s.alphas[j] - o_s.alphas[i])
            H = min(o_s.C, o_s.C + o_s.alphas[j] - o_s.alphas[i])
        else:
            L = max(0, o_s.alphas[i] + o_s.alphas[j] - o_s.C)
            H = min(o_s.C, o_s.alphas[j] + o_s.alphas[i])
        if L==H:
            print("L==H")
            return 0
        # 用核函数替代内积
        eta = 2.0 * o_s.K[i,j] - o_s.K[i,i] - o_s.K[j,j]
        if eta >= 0:
            print("eta >= 0")
            return 0
        o_s.alphas[j] -= o_s.label_mat[j] * (e_i - e_j) / eta
        o_s.alphas[j] = clip_alpha(o_s.alphas[j], H, L)
        update_e_k(o_s, j) # 更新误差缓存
        if abs(o_s.alphas[j] - alpha_j_old) < 0.00001:
            print("j not moving enough")
            return 0
        # 同时修改i, j但是方向相反
        o_s.alphas[i] += o_s.label_mat[j] * o_s.label_mat[i] * (alpha_j_old - o_s.alphas[j])
        update_e_k(o_s, i) #更新误差缓存
        # 设置常数项
        # 用核函数替代内积
        b1 = o_s.b - e_i - o_s.label_mat[i]*(o_s.alphas[i] - alpha_i_old) * o_s.K[i,i] \
            - o_s.label_mat[j]*(o_s.alphas[j] - alpha_j_old) * o_s.K[i,j]
        b2 = o_s.b - e_j - o_s.label_mat[i]*(o_s.alphas[i] - alpha_i_old) * o_s.K[i,j] \
                - o_s.label_mat[j]*(o_s.alphas[j] - alpha_j_old) * o_s.K[j,j]
        if 0 < o_s.alphas[i] and o_s.C > o_s.alphas[i]:
            o_s.b = b1
        elif 0 < o_s.alphas[j] and o_s.C > o_s.alphas[j]:
            o_s.b = b2
        else:
            o_s.b = (b1 + b2) / 2.0
        return 1
    else:
        return 0

测试

In [21]:
k1 = 1.3
rbf_data_arr, rbf_label_arr = load_data_set('data/testSetRBF.txt')
rbf_b, rbf_alphas = smo_p(rbf_data_arr, rbf_label_arr, 200, 0.0001, 10000, ('rbf', k1))

fullSet, iter: 0 i: 0, pair changed 1
fullSet, iter: 0 i: 1, pair changed 1
fullSet, iter: 0 i: 2, pair changed 2
fullSet, iter: 0 i: 3, pair changed 3
fullSet, iter: 0 i: 4, pair changed 3
fullSet, iter: 0 i: 5, pair changed 4
fullSet, iter: 0 i: 6, pair changed 5
fullSet, iter: 0 i: 7, pair changed 5
fullSet, iter: 0 i: 8, pair changed 6
fullSet, iter: 0 i: 9, pair changed 6
fullSet, iter: 0 i: 10, pair changed 7
fullSet, iter: 0 i: 11, pair changed 7
fullSet, iter: 0 i: 12, pair changed 7
fullSet, iter: 0 i: 13, pair changed 8
fullSet, iter: 0 i: 14, pair changed 9
fullSet, iter: 0 i: 15, pair changed 10
fullSet, iter: 0 i: 16, pair changed 11
fullSet, iter: 0 i: 17, pair changed 11
fullSet, iter: 0 i: 18, pair changed 12
fullSet, iter: 0 i: 19, pair changed 13
fullSet, iter: 0 i: 20, pair changed 13
fullSet, iter: 0 i: 21, pair changed 14
fullSet, iter: 0 i: 22, pair changed 14
fullSet, iter: 0 i: 23, pair changed 14
fullSet, iter: 0 i: 24, pair changed 15
j not moving enough
fullS

In [22]:
rbf_data_mat, rbf_label_mat = np.mat(rbf_data_arr), np.mat(rbf_label_arr).transpose()
sv_ind = np.nonzero(rbf_alphas> 0)[0]
svs = rbf_data_mat[sv_ind]
label_sv = rbf_label_mat[sv_ind]
print("There are %d Support Vectors" % np.shape(svs)[0])
def predict_error_rate(data_mat, label_arr):
    m, n = np.shape(data_mat)
    error_count = 0
    for i in range(m):
        kernel_eval = kernel_trans(svs, data_mat[i,:],('rbf',k1))
        predict = kernel_eval.T * np.multiply(label_sv, rbf_alphas[sv_ind]) + rbf_b
        if np.sign(predict) != np.sign(label_arr[i]):
            error_count += 1
    return float(error_count) / m

print("the training error rate is: %f" % predict_error_rate(rbf_data_mat, rbf_label_arr))
test_data_arr, test_label_arr = load_data_set('data/testSetRBF2.txt')
test_data_mat = np.mat(test_data_arr)
print("the test error rate is: %f" % predict_error_rate(test_data_mat, test_label_arr))   

There are 23 Support Vectors
the training error rate is: 0.020000
the test error rate is: 0.090000
